
**Librerías a usar**



In [990]:
from google.colab import drive

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os

import csv
from math import radians, sin, cos, sqrt, atan2
import json

from tqdm import tqdm
tqdm.pandas()

import warnings
warnings.filterwarnings('ignore')

In [991]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import ElasticNet
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_absolute_percentage_error

In [992]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [993]:
os.chdir(os.getcwd())

**Cargue de insumos**

In [994]:
puntos_interes = pd.read_csv('PuntosInteres.csv', dtype=str, encoding='latin1', header=None, delimiter=';', low_memory=False)
puntos_interes.columns = ['longitud', 'latitud', 'categoria', 'nombre']

puntos_interes.head()

,longitud,latitud,categoria,nombre
0,-73.9224858,4.5294102,pharmacy,No hallado
1,-75.5560536,10.3961543,pharmacy,No hallado
2,-75.5610232,10.3942042,pharmacy,No hallado
3,-74.2064019,4.7153596,pharmacy,DroguerÐa Colsubsidio Funza
4,-75.57863,6.1842195,pharmacy,Farmacia Pasteur


In [995]:
train_data = pd.read_csv('train_precios_vivienda.csv', dtype=str, index_col=0, low_memory=False)
test_data = pd.read_csv('test_precios_vivienda.csv', delimiter=';', low_memory=False)

**Limpieza y preparación de datos**

In [996]:
# Se inicia por eliminar de ambos dataset las columnas que son descriptivas y aquellas relacionadas con registros inmobiliarios y catastrales que se considera no tienen incidencia en la estimación del valor total del avaluo
columnas_a_eliminar = ['direccion_inmueble_informe', 'descripcion_general_sector', 'perspectivas_de_valorizacion', 'actualidad_edificadora','comportamiento_oferta_demanda', 'descripcion_tipo_inmueble', 'descripcion_uso_inmueble',
'descripcion_clase_inmueble', 'observaciones_altura_permitida','observaciones_aislamiento_posterior', 'observaciones_aislamiento_lateral', 'observaciones_antejardin', 'observaciones_indice_ocupacion',
'observaciones_indice_construccion', 'observaciones_generales_inmueble', 'condicion_ph', 'observaciones_estructura', 'observaciones_dependencias', 'numero_garaje_1', 'numero_garaje_2', 'numero_garaje_3',
'numero_garaje_4', 'numero_garaje_5', 'matricula_garaje_1', 'matricula_garaje_2', 'matricula_garaje_3', 'matricula_garaje_4', 'matricula_garaje_5', 'numero_deposito_1', 'matricula_inmobiliaria_deposito_1',
'numero_deposito_2', 'matricula_inmobiliaria_deposito_2', 'numero_deposito_3', 'matricula_inmobiliaria_deposito_3', 'numero_deposito_4', 'matricula_inmobiliaria_deposito_4', 'numero_deposito_5','uso_principal_ph',
'matricula_inmobiliaria_deposito_5', 'observaciones_generales_construccion', 'metodo_valuacion_1', 'metodo_valuacion_1', 'concepto_del_metodo_1', 'metodo_valuacion_2','concepto_del_metodo_2','metodo_valuacion_3',
'concepto_del_metodo_3','metodo_valuacion_4','concepto_del_metodo_4','metodo_valuacion_5', 'concepto_del_metodo_5', 'metodo_valuacion_6', 'concepto_del_metodo_6', 'metodo_valuacion_7', 'concepto_del_metodo_7',
'metodo_valuacion_8', 'concepto_del_metodo_8','metodo_valuacion_9', 'concepto_del_metodo_9', 'barrio', 'unidades', 'garaje_paralelo_4', 'garaje_doble_5', 'garaje_paralelo_5', 'garaje_servidumbre_5', 'fecha_aprobación', 'valor_uvr', 'valor_avaluo_en_uvr',
'ocupante','altura_permitida','aislamiento_lateral','aislamiento_posterior','indice_construccion','detalle_material','uso_principal_ph','area_actividad','indice_ocupacion']

for columna in columnas_a_eliminar:
    if columna in train_data.columns:
        train_data.drop(columna, axis=1, inplace=True)
    if columna in test_data.columns:
        test_data.drop(columna, axis=1, inplace=True)

In [997]:
# En segundo lugar, se hace limpieza de acentos en ambos dataset
diccionario_acentos = {
    'Ã¡': 'á',
    'Ã©': 'é',
    'Ã­': 'í',
    'Ã\xad': 'í',
    'Ã³': 'ó',
    'Ãº': 'ú',
    'Ã¼': 'ü',
    'Ã\x81': 'Á',
    'Ã‰': 'É',
    'Ã\x8d': 'Í',
    'Ã“': 'Ó',
    'Ãš': 'Ú',
    'Ãœ': 'Ü',
    'Ã±': 'ñ',
    'Ã‘':'Ñ'}
train_data.replace(diccionario_acentos, regex=True, inplace=True)
test_data.replace(diccionario_acentos, regex=True, inplace=True)

In [998]:
# En tercer lugar, para las columnas restantes que son de tipo de texto, se depura aquello que no corresponda a los datos esperados en la columna.

# Para esto, se empieza por unir temporalmente los dataframe en sus columnas comunes a fin de hacer una única revisión de los valores posibles en cada columna
df_concatenado = pd.concat([train_data, test_data], ignore_index=True)

valores_unicos_por_columna = {}

for columna in df_concatenado.columns:
    valores_unicos_por_columna[columna] = df_concatenado[columna].unique()

# Imprimir los valores únicos por columna
for columna, valores_unicos in valores_unicos_por_columna.items():
    print(f'Valores únicos en la columna "{columna}":')
    print(valores_unicos)
    print('\n')

Valores únicos en la columna "id":
['5896' '10570' '14600' ... 8095 7435 16318]


Valores únicos en la columna "objeto":
['Remate' 'Originación' '2018-06-19 19:48:27.269-05']


Valores únicos en la columna "motivo":
['Remates' 'Crédito hipotecario de vivienda' 'Colomext Hipotecario'
 'Empleados' 'Leasing Habitacional' 'Garantía' 'Compra de cartera'
 'Crédito Comercial' 'Actualización de garantías' 'Dación en Pago'
 'Hipotecario Visto Bueno' 'Leasing Visto Bueno'
 'Leasing Inmobiliario - Persona Natural' '0' 'Leasing Comercial'
 'Originación' 'Reformas']


Valores únicos en la columna "proposito":
['Garantía Hipotecaria' 'Valor Asegurable'
 'Transacción Comercial de Venta' '0' 'Crédito hipotecario de vivienda']


Valores únicos en la columna "tipo_avaluo":
['Remates' 'Hipotecario' 'Garantía Hipotecaria']


Valores únicos en la columna "tipo_credito":
['Vivienda' 'Diferente de Vivienda' '0' 'Hipotecario']


Valores únicos en la columna "tipo_subsidio":
[nan 'VIS' 'Frech II' 'VIP' 'Gratui

In [999]:
# Iniciando con las columnas que corresponden a estado o calidad, dado que tienen criterios específicos por lo que se pueden eliminar todos los registros que no cumplen estos criterios
list(df_concatenado['estado_acabados_pisos'].unique())

['Bueno',
 'Sin acabados',
 'Regular',
 '"Espacios funcionales ventilados e iluminados."',
 '"Primer piso: sala, comedor, cuatro alcobas, cocina, un baño y zona de ropas. Segundo piso (nivel inferior): dos alcobas, un baño, cocina, patio descubierto y solar."',
 '"El inmueble goza en general de buena distribución e iluminación. su distribución así: sala, comedor, cocina, zona de ropas,  baño social, 3 alcobas,  espacio para futuro baño"',
 '"Hay buena distribución de dependencias, con espacios amplios."',
 '0',
 '"Buena distribucion, ventilacion e iluminacion natural. El Antejardin es utilizado como garaje de motos."',
 '"La unidad que conforma la vivienda cuenta con dependencias necesarias para el uso al que se encuentra sometido. Al igual de manera básica presentan funcionalidad sobre el inmueble mismo. La unidad compuesta por el apartamento solo alcoba y baño. "',
 '"Buena distribución de las dependencias. "',
 '"Bien inmueble con una adecuada distribución. "',
 '"el inmueble cuenta

In [1000]:
criterios_estado_piso = ['Bueno', 'Sin acabados', 'Regular', 'Malo']
train_data.drop(train_data[~train_data['estado_acabados_pisos'].isin(criterios_estado_piso)].index, inplace=True)
test_data.drop(test_data[~test_data['estado_acabados_pisos'].isin(criterios_estado_piso)].index, inplace=True)

In [1001]:
list(df_concatenado['calidad_acabados_pisos'].unique())

['Normal',
 'Lujoso',
 'Sin Acabados',
 'Sencillo',
 'Bueno',
 '0',
 '"dependencias de buen tamaño y distribución, estado de conservación aceptable la habitación del primer piso fue hecha recientemente con divisiones en driwall. "',
 'y en 3 piso un apartamento con sala, comedor, cocina, ropas, baño y 2 alcobas."',
 '"Buena distribución de las dependencias. "',
 '"Las dependencias se encuentran en buen estado al momento de la visita"',
 '"Buena distribución de las dependencias."',
 '"El inmueble tiene una buena distribución interior , con buena iluminación y ventilación."',
 'tercer piso, aún no se puede preci"',
 '"Se observa con buena distribución todos los espacios."',
 'Piso 3: Alcoba con baño privado y terraza. Las dependencias cuenta con buena iluminación  ventilación natural. "',
 '"Sala comedor, balcón, cocina tipo americano, cuarto de ropas con lavadero, baño social, dos habitaciones, una es la principal con baño privado. Piso en Porcelanato. Construcción Bloque H5. Vivienda N

In [1002]:
criterios_acabados_piso = ['Normal', 'Lujoso', 'Sin Acabados', 'Sencillo', 'Bueno', 'Regular', 'Sin acabados']
train_data.drop(train_data[~train_data['calidad_acabados_pisos'].isin(criterios_acabados_piso)].index, inplace=True)
test_data.drop(test_data[~test_data['calidad_acabados_pisos'].isin(criterios_acabados_piso)].index, inplace=True)

train_data['calidad_acabados_pisos'] = train_data['calidad_acabados_pisos'].replace('Sin Acabados','Sin acabados')
test_data['calidad_acabados_pisos'] = test_data['calidad_acabados_pisos'].replace('Sin Acabados','Sin acabados')

In [1003]:
list(df_concatenado['estado_acabados_muros'].unique())

['Bueno',
 'Sin acabados',
 'Regular',
 'Normal',
 'Sencillo',
 '0',
 'Dependencias estrechas por su forma alargada pero bien iluminadas por la serie de vacíos que tiene.',
 'Lujoso',
 'Malo',
 '"El inmueble cuenta con buena distribución espacial."',
 '"El apartamento 307, consta de: sala, comedor, balcón, cocina, zona de oficios, estar, baño social, alcoba auxiliar y alcoba  principal con baño privado. "',
 '"El inmueble tasado corresponde a una casa tipo VIS, medianera, de 2 pisos, estrato 2. Adecuada distribución: 1er piso con: jardín con garaje, sala comedor, cocina, patio ropas, baño y 1 alcoba',
 '"Buena distribución de las dependencias. "',
 '"El apartamento 304 del edificio 4, consta de: sala, comedor, balcón, cocina, zona de oficios, estudio, baño social, 2 alcobas auxiliares y alcoba principal con baño privado y vestier"',
 '"Bien inmueble con una adecuada distribución."',
 'Sin Acabados',
 '"Las dependencias de la vivienda cuentas con unas áreas producto de dimensiones aptas

In [1004]:
criterios_acabados_muros = ['Bueno', 'Sin acabados', 'Regular', 'Normal', 'Sencillo','Lujoso', 'Malo', 'Sin Acabados']
train_data.drop(train_data[~train_data['estado_acabados_muros'].isin(criterios_acabados_muros)].index, inplace=True)
test_data.drop(test_data[~test_data['estado_acabados_muros'].isin(criterios_acabados_muros)].index, inplace=True)

train_data['estado_acabados_muros'] = train_data['estado_acabados_muros'].replace('Sin Acabados','Sin acabados')
test_data['estado_acabados_muros'] = test_data['estado_acabados_muros'].replace('Sin Acabados','Sin acabados')

In [1005]:
list(df_concatenado['calidad_acabados_muros'].unique())

['Normal',
 'Sin Acabados',
 'Lujoso',
 'Sencillo',
 'Bueno',
 '"Solo el baño social se entrega con aparatos instalados"',
 '0',
 '2do piso comprende: 2 alcobas, hall y baño de alcobas."',
 'Regular',
 'Sin acabados']

In [1006]:
criterios_calidad_muros = ['Normal', 'Sin acabados', 'Lujoso', 'Sencillo', 'Bueno','Regular', 'Sin Acabados']
train_data.drop(train_data[~train_data['calidad_acabados_muros'].isin(criterios_calidad_muros)].index, inplace=True)
test_data.drop(test_data[~test_data['calidad_acabados_muros'].isin(criterios_calidad_muros)].index, inplace=True)

train_data['calidad_acabados_muros'] = train_data['calidad_acabados_muros'].replace('Sin Acabados','Sin acabados')
test_data['calidad_acabados_muros'] = test_data['calidad_acabados_muros'].replace('Sin Acabados','Sin acabados')

In [1007]:
list(df_concatenado['estado_acabados_techos'].unique())

['Bueno',
 'Sin acabados',
 'Regular',
 'Malo',
 'Normal',
 'Sencillo',
 '"Ante jardín sin cerramiento',
 'Lujoso',
 '0',
 'Sin Acabados',
 '"Buena distribución de las dependencias."']

In [1008]:
criterios_estado_techos = ['Bueno', 'Sin acabados', 'Regular', 'Malo', 'Normal','Sencillo', 'Lujoso',  'Sin Acabados']
train_data.drop(train_data[~train_data['estado_acabados_techos'].isin(criterios_estado_techos)].index, inplace=True)
test_data.drop(test_data[~test_data['estado_acabados_techos'].isin(criterios_estado_techos)].index, inplace=True)

train_data['estado_acabados_techos'] = train_data['estado_acabados_techos'].replace('Sin Acabados','Sin acabados')
test_data['estado_acabados_techos'] = test_data['estado_acabados_techos'].replace('Sin Acabados','Sin acabados')

In [1009]:
list(df_concatenado['calidad_acabados_techos'].unique())

['Normal',
 'Lujoso',
 'Sin Acabados',
 'Sencillo',
 'Bueno',
 'Sala comedor, cocina sencilla con mesón en concreto, baño social, dos habitaciones, patio de ropas con lavadero. "',
 '"Dependencias amplias en sala, comedor, estudio y terrazas',
 'Regular',
 'Sin acabados',
 '0']

In [1010]:
criterios_calidad_techos = ['Normal', 'Lujoso', 'Sin Acabados', 'Sencillo', 'Bueno', 'Regular', 'Sin acabados']
train_data.drop(train_data[~train_data['calidad_acabados_techos'].isin(criterios_calidad_techos)].index, inplace=True)
test_data.drop(test_data[~test_data['calidad_acabados_techos'].isin(criterios_calidad_techos)].index, inplace=True)

train_data['calidad_acabados_techos'] = train_data['calidad_acabados_techos'].replace('Sin Acabados','Sin acabados')
test_data['calidad_acabados_techos'] = test_data['calidad_acabados_techos'].replace('Sin Acabados','Sin acabados')

In [1011]:
df_concatenado['estado_acabados_madera'].unique()

array(['Bueno', 'Sin acabados', 'Regular', 'Normal', 'Sencillo', 'Malo',
       'Lujoso',
       'pero las habitaciones son muy estrechas o con dimensiones mal diseñadas que hacen que el inmueble no sea atractivo para vivir."',
       'Sin Acabados',
       '"Consta de: sala, comedor, balcón, cocina integral abierta con mezón tipo barra americana de Quarzón, dos alcobas auxiliares, baño social y alcoba principal con baño privado'],
      dtype=object)

In [1012]:
criterios_estado_madera = ['Bueno', 'Sin acabados', 'Regular', 'Normal', 'Sencillo', 'Malo', 'Lujoso', 'Sin Acabados']
train_data.drop(train_data[~train_data['estado_acabados_madera'].isin(criterios_estado_madera)].index, inplace=True)
test_data.drop(test_data[~test_data['estado_acabados_madera'].isin(criterios_estado_madera)].index, inplace=True)

train_data['estado_acabados_madera'] = train_data['estado_acabados_madera'].replace('Sin Acabados','Sin acabados')
test_data['estado_acabados_madera'] = test_data['estado_acabados_madera'].replace('Sin Acabados','Sin acabados')

In [1013]:
df_concatenado['calidad_acabados_madera'].unique()

array(['Normal', 'Sin Acabados', 'Lujoso', 'Sencillo', 'Bueno',
       'Sin acabados', 'Malo', 'Regular', 'garaje cubierto."'],
      dtype=object)

In [1014]:
criterios_calidad_madera = ['Normal', 'Sin Acabados', 'Lujoso', 'Sencillo', 'Bueno', 'Sin acabados', 'Malo', 'Regular']
train_data.drop(train_data[~train_data['calidad_acabados_madera'].isin(criterios_calidad_madera)].index, inplace=True)
test_data.drop(test_data[~test_data['calidad_acabados_madera'].isin(criterios_calidad_madera)].index, inplace=True)

train_data['calidad_acabados_madera'] = train_data['calidad_acabados_madera'].replace('Sin Acabados','Sin acabados')
test_data['calidad_acabados_madera'] = test_data['calidad_acabados_madera'].replace('Sin Acabados','Sin acabados')

In [1015]:
list(df_concatenado['estado_acabados_metal'].unique())

['Bueno',
 'Sin acabados',
 'Regular',
 'Normal',
 'Sencillo',
 'Malo',
 'Lujoso',
 'Sin Acabados']

In [1016]:
train_data['estado_acabados_metal'] = train_data['estado_acabados_metal'].replace('Sin Acabados','Sin acabados')
test_data['estado_acabados_metal'] = test_data['estado_acabados_metal'].replace('Sin Acabados','Sin acabados')

In [1017]:
list(df_concatenado['calidad_acabados_metal'].unique())

['Normal', 'Sencillo', 'Lujoso', 'Sin Acabados', 'Bueno', 'Regular']

In [1018]:
train_data['calidad_acabados_metal'] = train_data['calidad_acabados_metal'].replace('Sin Acabados','Sin acabados')
test_data['calidad_acabados_metal'] = test_data['calidad_acabados_metal'].replace('Sin Acabados','Sin acabados')

In [1019]:
list(df_concatenado['estado_acabados_banos'].unique())

['Bueno',
 'Sin acabados',
 'Regular',
 'Normal',
 'Sencillo',
 'Malo',
 'Lujoso',
 'Sin Acabados']

In [1020]:
train_data['estado_acabados_banos'] = train_data['estado_acabados_banos'].replace('Sin Acabados','Sin acabados')
test_data['estado_acabados_banos'] = test_data['estado_acabados_banos'].replace('Sin Acabados','Sin acabados')

In [1021]:
list(df_concatenado['calidad_acabados_banos'].unique())

['Normal', 'Lujoso', 'Sencillo', 'Sin Acabados', 'Bueno', 'Regular']

In [1022]:
train_data['calidad_acabados_banos'] = train_data['calidad_acabados_banos'].replace('Sin Acabados','Sin acabados')
test_data['calidad_acabados_banos'] = test_data['calidad_acabados_banos'].replace('Sin Acabados','Sin acabados')

In [1023]:
list(df_concatenado['estado_acabados_cocina'].unique())

['Bueno', 'Sin acabados', 'Regular', 'Normal', 'Sencillo', 'Malo', 'Lujoso']

In [1024]:
train_data['estado_acabados_cocina'] = train_data['estado_acabados_cocina'].replace('Sin Acabados','Sin acabados')
test_data['estado_acabados_cocina'] = test_data['estado_acabados_cocina'].replace('Sin Acabados','Sin acabados')

In [1025]:
list(df_concatenado['calidad_acabados_cocina'].unique())

['Integral',
 'Sencillo',
 'Semi-Integral',
 'Sin Acabados',
 'Bueno',
 'Normal',
 'Regular',
 'Lujoso']

In [1026]:
train_data['calidad_acabados_cocina'] = train_data['calidad_acabados_cocina'].replace('Sin Acabados','Sin acabados')
test_data['calidad_acabados_cocina'] = test_data['calidad_acabados_cocina'].replace('Sin Acabados','Sin acabados')

In [1027]:
# Continuamos con las columnas que son binarias (Si, No) para darles un formato único
diccionario_binarios = {'No':0, 'Si':1, '0':0, '1':1, 'No Aplica':0, 'Aplica':1}

In [1028]:
columnas_binarias = ['sometido_a_propiedad_horizontal','antejardin','alcantarillado_en_el_predio' ,
'acueducto_en_el_predio' ,
'gas_en_el_predio' ,
'energia_en_el_predio' ,
'telefono_en_el_predio' ,
'predio_subdividido_fisicamente' ,
'rph' ,
'porteria' ,
'citofono' ,
'bicicletero' ,
'piscina' ,
'tanque_de_agua' ,
'club_house' ,
'garaje_visitantes' ,
'teatrino' ,
'sauna' ,
'vigilancia_privada' ,
'administracion' ,
'garaje_doble_1' ,
'garaje_paralelo_1' ,
'garaje_servidumbre_1' ,
'garaje_cubierto_2' ,
'garaje_doble_2' ,
'garaje_paralelo_2' ,
'garaje_servidumbre_2' ,
'garaje_cubierto_3' ,
'garaje_doble_3' ,
'garaje_paralelo_3' ,
'garaje_servidumbre_3' ,
'garaje_cubierto_4' ,
'garaje_doble_4' ,
'garaje_servidumbre_4' ,
'garaje_cubierto_5' ,
'area_libre' ,
'barrio_legal',
'alcantarillado_en_el_sector',
'acueducto_en_el_sector',
'gas_en_el_sector',
'energia_en_el_sector',
'telefono_en_el_sector',
'vias_pavimentadas',
'sardineles_en_las_vias',
'andenes_en_las_vias',
'paradero',
'alumbrado',
'arborizacion',
'alamedas',
'ciclo_rutas']

for columna in columnas_binarias:
    train_data[columna] = train_data[columna].replace(diccionario_binarios)
    test_data[columna] = test_data[columna].replace(diccionario_binarios)


In [1029]:
df_concatenado['garaje_cubierto_1'].unique()

array(['0', 'Si', 'No', '370-926861', '370-80382', '50N-20030335',
       'Sencillo', '1', '001-903669', '018 - 140475', '040 - 537432',
       '370-780052', '38', '67', '2', '370-330832', '43044', '001-842004',
       'E-203', '060-233963', 'Malo', '001-1053647', '12905',
       '50N-941945', '50C-1731779', '001-547629', '161', '240-269872',
       '28', '001-1115065', '001-625912', '001-806034', '50C-1127807',
       'No Tiene', '216', 'Normal', '370-955576', '060-88779'],
      dtype=object)

In [1030]:
train_data.drop(train_data[train_data['garaje_cubierto_1'] == '50C-1731779'].index, inplace=True)
test_data.drop(test_data[test_data['garaje_cubierto_1'] == '50C-1731779'].index, inplace=True)

train_data['garaje_cubierto_1'] = train_data['garaje_cubierto_1'].replace(diccionario_binarios)
test_data['garaje_cubierto_1'] = test_data['garaje_cubierto_1'].replace(diccionario_binarios)

In [1031]:
df_concatenado['tipo_vigilancia'].unique()

array(['0', '24 Horas', '12 Horas', 'No', 'Si'], dtype=object)

In [1032]:
diccionario_vigilancia={'24 Horas':1,'0':0,'12 Horas':1}
train_data['tipo_vigilancia'] = train_data['tipo_vigilancia'].replace(diccionario_vigilancia)
test_data['tipo_vigilancia'] = test_data['tipo_vigilancia'].replace(diccionario_vigilancia)

In [1033]:
columnas_con_cero = ['proposito', 'tipo_credito', 'tipo_inmueble', 'motivo', 'uso_actual', 'tipo_deposito']

for columna in columnas_con_cero:
    train_data.drop(train_data[train_data[columna] == '0'].index, inplace=True)
    test_data.drop(test_data[test_data[columna] == '0'].index, inplace=True)

In [1034]:
columnas_no_disponible = ['danos_previos', 'irregularidad_altura', 'irregularidad_planta', 'ajustes_sismoresistentes', 'estructura_reforzada']

for columna in columnas_no_disponible:
    train_data.drop(train_data[train_data[columna] == 'No disponible'].index, inplace=True)
    test_data.drop(test_data[test_data[columna] == 'No disponible'].index, inplace=True)


In [1035]:
diccionario_danos = {'Con daños previos':1,'Sin daños previos':0}
train_data['danos_previos'] = train_data['danos_previos'].replace(diccionario_danos)
test_data['danos_previos'] = test_data['danos_previos'].replace(diccionario_danos)

In [1036]:
diccionario_ajustes = {'Reparados':1,'No Reparados':0}
train_data['ajustes_sismoresistentes'] = train_data['ajustes_sismoresistentes'].replace(diccionario_ajustes)
test_data['ajustes_sismoresistentes'] = test_data['ajustes_sismoresistentes'].replace(diccionario_ajustes)

In [1037]:
diccionario_irregularidad = {'Sin irregularidad':0, 'Con irregularidad':1}
columnas_irregularidad = ['irregularidad_planta', 'irregularidad_altura']

for columna in columnas_irregularidad:
    train_data[columna] = train_data[columna].replace(diccionario_irregularidad)
    test_data[columna] = test_data[columna].replace(diccionario_irregularidad)

In [1038]:
df_concatenado = pd.concat([train_data, test_data], ignore_index=True)

valores_unicos_por_columna = {}

for columna in df_concatenado.columns:
    valores_unicos_por_columna[columna] = df_concatenado[columna].unique()

# Imprimir los valores únicos por columna
for columna, valores_unicos in valores_unicos_por_columna.items():
    print(f'Valores únicos en la columna "{columna}":')
    print(valores_unicos)
    print('\n')

Valores únicos en la columna "id":
['9101' '6770' '3343' '8209' '5874' '2054' '8568' '3525' '2051' '9652'
 '4793' '7495' '828' '4230' '1029' '8704' '4224' '2909' '7074' '11143'
 '3703' '5411' '630' '13819' '4365' '11900' '4964' '5199' '3560' '1560'
 '6894' '4125' '4194' '4557' '3174' '694' '5621' '1057' '6233' '6247'
 '8489' '8470' '6406' '8360' '4243' '1952' '5522' '3361' '7117' '708'
 '8452' '4562' '4566' '1961' '4619' '4653' '8603' '3004' '3753' '2114'
 '486' '1941' '2459' '10073' '6358' '15057' '4632' '2432' '11519' '12863'
 '4659' '16702' '6070' '4925' '5167' '12269' '3913' '5005' '6140' '175'
 '4906' '3900' '4011' '4941' '4277' '2988' '4936' '5730' '1387' 3961 6780
 4094 549]


Valores únicos en la columna "objeto":
['Originación' 'Remate']


Valores únicos en la columna "motivo":
['Leasing Habitacional' 'Crédito hipotecario de vivienda'
 'Compra de cartera' 'Garantía' 'Remates' 'Actualización de garantías'
 'Empleados' 'Dación en Pago']


Valores únicos en la columna "proposito"

In [1039]:
columnas_numericas = ['accesorios',
'acueducto_en_el_predio',
'acueducto_en_el_sector',
'administracion',
'ajustes_sismoresistentes',
'alamedas',
'alcantarillado_en_el_predio',
'alcantarillado_en_el_sector',
'alumbrado',
'andenes_en_las_vias',
'antejardin',
'arborizacion',
'area_construccion',
'area_deposito',
'area_garaje',
'area_libre',
'area_otros',
'area_privada',
'area_terreno',
'area_valorada',
'balcon',
'bano_privado',
'bano_servicio',
'bano_social',
'barrio_legal',
'bicicletero',
'bodega',
'ciclo_rutas',
'citofono',
'closet',
'club_house',
'cocina',
'comedor',
'contadores_agua',
'contadores_luz',
'cuarto_servicio',
'danos_previos',
'energia_en_el_predio',
'energia_en_el_sector',
'estar_habitacion',
'estudio',
'garaje_cubierto_1',
'garaje_cubierto_2',
'garaje_cubierto_3',
'garaje_cubierto_4',
'garaje_cubierto_5',
'garaje_doble_1',
'garaje_doble_2',
'garaje_doble_3',
'garaje_doble_4',
'garaje_paralelo_1',
'garaje_paralelo_2',
'garaje_paralelo_3',
'garaje_servidumbre_1',
'garaje_servidumbre_2',
'garaje_servidumbre_3',
'garaje_servidumbre_4',
'garaje_visitantes',
'gas_en_el_predio',
'gas_en_el_sector',
'habitaciones',
'id',
'indice_ocupacion',
'irregularidad_altura',
'irregularidad_planta',
'jardin',
'Latitud',
'local',
'Longitud',
'numero_de_edificios',
'numero_piso',
'numero_total_de_garajes',
'numero_total_depositos',
'oficina',
'paradero',
'patio_interior',
'piscina',
'pisos_bodega',
'porteria',
'predio_subdividido_fisicamente',
'rph',
'sala',
'sardineles_en_las_vias',
'sauna',
'sometido_a_propiedad_horizontal',
'tanque_de_agua',
'teatrino',
'telefono_en_el_predio',
'telefono_en_el_sector',
'terraza',
'tipo_vigilancia',
'total_cupos_parquedaro',
'valor_area_construccion',
'valor_area_deposito',
'valor_area_garaje',
'valor_area_libre',
'valor_area_otros',
'valor_area_privada',
'valor_area_terreno',
'valor_total_avaluo',
'vetustez',
'vias_pavimentadas',
'vigilancia_privada',
'zona_de_ropas',
'zona_verde_privada',
]

for columna in columnas_numericas:
    if columna in train_data.columns:
        train_data[columna]=train_data[columna].apply(pd.to_numeric, errors='coerce')
    if columna in test_data.columns:
        test_data[columna]=test_data[columna].apply(pd.to_numeric, errors='coerce')

**Definir tipo de variables**

In [1061]:
def clasificar_caracteristicas(dataframe, lista_categoricas, lista_numericas, umbral_unicos):
    if not lista_categoricas or not lista_numericas:
        raise ValueError("Las listas de características categóricas y numéricas no deben estar vacías.")

    caracteristicas_categoricas = []
    caracteristicas_numericas = []

    for columna in dataframe.columns:
        if columna not in ['valor_total_avaluo', 'id']:
            if (len(dataframe[columna].unique()) <= umbral_unicos or columna in lista_categoricas) and columna not in lista_numericas:
                caracteristicas_categoricas.append(columna)
            elif (len(dataframe[columna].unique()) > umbral_unicos or columna in lista_numericas) and columna not in lista_categoricas:
                caracteristicas_numericas.append(columna)

    return caracteristicas_categoricas, caracteristicas_numericas

In [1060]:
carac_categoricas = [
    'departamento_inmueble', 'municipio_inmueble'
]
carac_numericas = [
    'habitaciones', 'closet', 'bano_privado', 'sala', 'comedor', 'bano_social', 'cocina', 'zona_de_ropas', 'contadores_agua', 'accesorios',
    'patio_interior', 'contadores_luz', 'balcon', 'terraza', 'oficina', 'total_cupos_parquedaro', 'estrato', 'bano_servicio', 'numero_total_de_garajes',
    'local', 'estar_habitacion', 'jardin', 'numero_total_depositos', 'cuarto_servicio', 'pisos_bodega', 'bodega'
]
threshold_unique = 40

caracteristicas_categoricas, caracteristicas_numericas = clasificar_caracteristicas(train_data, carac_categoricas, carac_numericas, threshold_unique)

print("Características categóricas:")
print(caracteristicas_categoricas)

print("\nCaracterísticas numéricas:")
print(caracteristicas_numericas)

Características categóricas:
['objeto', 'motivo', 'proposito', 'tipo_avaluo', 'tipo_credito', 'tipo_subsidio', 'departamento_inmueble', 'municipio_inmueble', 'sector', 'alcantarillado_en_el_sector', 'acueducto_en_el_sector', 'gas_en_el_sector', 'energia_en_el_sector', 'telefono_en_el_sector', 'vias_pavimentadas', 'sardineles_en_las_vias', 'andenes_en_las_vias', 'barrio_legal', 'topografia_sector', 'condiciones_salubridad', 'transporte', 'demanda_interes', 'paradero', 'alumbrado', 'arborizacion', 'alamedas', 'ciclo_rutas', 'nivel_equipamiento_comercial', 'alcantarillado_en_el_predio', 'acueducto_en_el_predio', 'gas_en_el_predio', 'energia_en_el_predio', 'telefono_en_el_predio', 'tipo_inmueble', 'uso_actual', 'clase_inmueble', 'sometido_a_propiedad_horizontal', 'antejardin', 'predio_subdividido_fisicamente', 'area_valorada', 'numero_piso', 'numero_de_edificios', 'rph', 'porteria', 'citofono', 'bicicletero', 'piscina', 'tanque_de_agua', 'club_house', 'garaje_visitantes', 'teatrino', 'saun

In [1062]:
categorica = [x[0] for x in caracteristicas_categoricas]
numerica = [x[0] for x in caracteristicas_numericas]

In [1063]:
for col in caracteristicas_numericas:
    try:
        # Intenta convertir directamente a float
        test_data[col] = test_data[col].astype(float)
        train_data[col] = train_data[col].astype(float)
    except KeyError as e:
        # Imprime información de depuración
        print(f"Error en la columna {col}: {e}")

        # En caso de error, reemplaza comas por puntos y convierte a float
        test_data[col] = test_data[col].str.replace(',', '.')
        test_data[col] = pd.to_numeric(test_data[col], errors='coerce').fillna(0).astype(float)

        train_data[col] = train_data[col].str.replace(',', '.')
        train_data[col] = pd.to_numeric(train_data[col], errors='coerce').fillna(0).astype(float)

# Convertir de nuevo a str después de la manipulación
for col in caracteristicas_numericas:
    test_data[col] = test_data[col].astype(str)

In [1064]:
caracteristicas = [x for x in train_data.columns if x not in ['valor_total_avaluo', 'id']]

X_train, y_train = train_data[caracteristicas], train_data['valor_total_avaluo']
y_train = y_train.astype(float)

In [1065]:
test_data['valor_total_avaluo'] = 0

X_test, y_test = test_data[caracteristicas], test_data['valor_total_avaluo']
y_test = pd.to_numeric(y_test, errors='coerce').fillna(0).astype(float)

In [1066]:
# Create a pipeline for categorical features
categorical_pipeline = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Create a pipeline for numerical features
numerical_pipeline = Pipeline([
    ('scaler', StandardScaler())
])

# Use ColumnTransformer to apply the appropriate preprocessing for each feature type
preprocessor = ColumnTransformer([
    ('categorical', categorical_pipeline, caracteristicas_categoricas),
    ('numerical', numerical_pipeline, caracteristicas_numericas)
])

# Fit and transform the training data
X_train_enc = preprocessor.fit_transform(X_train)

# Transform the test data
X_test_enc = preprocessor.transform(X_test[caracteristicas])

# Obtener los nombres de las características
feature_names = preprocessor.get_feature_names_out()

In [1067]:
# Eliminar filas con NaN en y_train
X_train = X_train[~y_train.isna()]
y_train = y_train.dropna()

In [1068]:
# Eliminar filas con NaN en y_test
X_test = X_test[~y_test.isna()]
y_test = y_test.dropna()

In [1069]:
# Ahora, realizar la búsqueda de hiperparámetros y evaluación del modelo
model_search_results = {}

# Perform random search for each model
for model_name, model in tqdm(models.items()):
    search = RandomizedSearchCV(model, param_distributions[model_name], n_iter=20,
                                scoring=make_scorer(mean_absolute_percentage_error, greater_is_better=False),
                                cv=5, n_jobs=-1, error_score='raise')

    try:
        search.fit(X_train_enc, y_train)

        # Evaluar en el conjunto de prueba después del ajuste
        y_pred = search.predict(X_test_enc)

        model_search_results[model_name] = {
            'best_params': search.best_params_,
            'test_MAPE': mean_absolute_percentage_error(y_test, y_pred),
            'model': search.best_estimator_  # Almacena el modelo ajustado
        }
    except Exception as e:
        print(f"Error during fitting {model_name}: {e}")

100%|██████████| 10/10 [00:00<00:00, 1059.27it/s]

Error during fitting Linear Regression: Found input variables with inconsistent numbers of samples: [89, 30]
Error during fitting Ridge Regression: Found input variables with inconsistent numbers of samples: [89, 30]
Error during fitting Lasso Regression: Found input variables with inconsistent numbers of samples: [89, 30]
Error during fitting ElasticNet Regression: Found input variables with inconsistent numbers of samples: [89, 30]
Error during fitting Decision Tree Regressor: Found input variables with inconsistent numbers of samples: [89, 30]
Error during fitting Random Forest Regressor: Found input variables with inconsistent numbers of samples: [89, 30]
Error during fitting Gradient Boosting Regressor: Found input variables with inconsistent numbers of samples: [89, 30]
Error during fitting Support Vector Regressor: Found input variables with inconsistent numbers of samples: [89, 30]
Error during fitting K-Nearest Neighbors Regressor: Found input variables with inconsistent numbe

In [1070]:
results = []
for key in model_search_results.keys():
    row = [key, model_search_results[key]['best_params'], model_search_results[key]['test_MAPE']]
    results.append(row)

df_results = pd.DataFrame(data=results, columns=['Model', 'Best Parameters', 'Test MAPE'])
df_results['Test MAPE'] = df_results['Test MAPE'].apply(lambda x: format(float(x), ".4f"))
df_results.sort_values('Test MAPE')

,Model,Best Parameters,Test MAPE
